# 🚀 Demo: Arquitetura Medallion no Databricks
Nesta demonstração, vamos simular um pipeline de dados básico utilizando os conceitos de **Camada Bronze** e **Camada Silver**.

**O que vamos fazer:**
1. **Ingestão:** Trazer os dados brutos para o ambiente Spark.
2. **Camada Bronze:** Salvar os dados originais com data de recebimento.
3. **Camada Silver:** Limpar, padronizar e preparar os dados para análise.

---

In [0]:
import pandas as pd
from pyspark.sql import functions as F

# 1. Link do Dataset (Catálogo de títulos)
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-20/netflix_titles.csv"

# 2. Leitura via Pandas (Útil para coletar dados externos via URL)
pdf = pd.read_csv(url)

# 3. Conversão para Spark DataFrame
# Transformamos em Spark para processar grandes volumes de dados de forma distribuída
df_raw = spark.createDataFrame(pdf)

print("✅ Dados carregados com sucesso no Spark!")

## 🥉 Camada Bronze (Raw)
A **Camada Bronze** armazena o dado bruto. 
- **Regra:** Não alteramos o conteúdo original.
- **Ação:** Apenas adicionamos uma coluna de controle para saber *quando* o dado foi coletado.

In [0]:
# Criar o banco de dados se não existir
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")

# Adicionamos uma coluna de controle (_ingestion_date) para auditoria
bronze_df = df_raw.withColumn("_ingestion_date", F.current_timestamp())

# Salvando no formato Delta (Padrão do Databricks)
bronze_df.write.format("delta").mode("overwrite").saveAsTable("bronze.plotly_raw")

print("✅ Tabela BRONZE criada!")
display(spark.table("bronze.plotly_raw").limit(5))

## 🥈 Camada Silver (Clean/Refined)
Na **Camada Silver**, os dados passam por uma "filtragem". O objetivo é deixar os dados prontos para os analistas de negócios.

**Transformações aplicadas:**
1. Padronização de nomes (snake_case e minúsculas).
2. Remoção de registros completamente vazios.

In [0]:
# 1. Preparação do ambiente
spark.sql("CREATE DATABASE IF NOT EXISTS silver")

# 2. Leitura da camada anterior (Bronze)
bronze_data = spark.table("bronze.plotly_raw")

# 3. Limpeza e Padronização
silver_df = bronze_data

# Loop para renomear colunas: remove espaços e coloca em minúsculas
for column in silver_df.columns:
    new_name = column.lower().replace(" ", "_")
    silver_df = silver_df.withColumnRenamed(column, new_name)

# Remove linhas onde todos os valores são nulos
silver_df = silver_df.dropna(how="all")

# 4. Escrita na Camada Silver
silver_df.write.format("delta").mode("overwrite").saveAsTable("silver.plotly_clean")

print("✅ Tabela SILVER processada e pronta para uso!")
display(spark.table("silver.plotly_clean").limit(10))